#Set-Up

In [2]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Problem A: Practice writing an optimizing problem
A rectangular box must have volume 500 in^3 (Recall: a box's volume is defined by its $length*width*height$).

Find the shape that has the smallest "mailing length" where the mailing length is defined as the sum of the three edge lengths.

What is the optimal shape of the box that meets this volume requirement with the smallest mailing length possible? (Use ipopt since this is a nonlinear problem).

In [15]:
#initialize a "Concrete Model"
modelA = ConcreteModel()
num_var =3
#initialize DVs
modelA.x = Var(range(num_var), bounds = (0.1,100))
#define the objective
modelA.Objective = Objective(expr=sum(modelA.x[i] for i in range(num_var)),sense=minimize)
#constraint
volume = 500
modelA.Volume_Constraint = Constraint(expr=modelA.x[0] * modelA.x[1] * modelA.x[2] == volume)
modelA.pprint()

1 Var Declarations
    x : Size=3, Index={0, 1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   0.1 :  None :   100 : False :  True :  Reals
          1 :   0.1 :  None :   100 : False :  True :  Reals
          2 :   0.1 :  None :   100 : False :  True :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + x[1] + x[2]

1 Constraint Declarations
    Volume_Constraint : Size=1, Index=None, Active=True
        Key  : Lower : Body           : Upper : Active
        None : 500.0 : x[0]*x[1]*x[2] : 500.0 :   True

3 Declarations: x Objective Volume_Constraint


In [17]:
#solve model (Note: you should use "ipopt" because this is a nonlinear problem)
opt = SolverFactory('ipopt')
results = opt.solve(modelA, tee = False) #setting tee = False hides the diagnostic outputs


In [18]:
#print optimal solution and the smallest mailing length it achieves
#print optimal solution and the smallest mailing length it achieves
print(f"x[0] (length)={modelA.x[0]():.4f} inches")
print(f"x[1] (width)={modelA.x[1]():.4f} inches")
print(f"x[2] (height)={modelA.x[2]():.4f} inches")
print(f"\nMinimum Mailing Length = {modelA.Objective():.4f} inches")


x[0] (length)=7.9370 inches
x[1] (width)=7.9370 inches
x[2] (height)=7.9370 inches

Minimum Mailing Length = 23.8110 inches


#Problem B: Optimizing an Existing Function (office building)
Below, I've included a completed "office building" model as a function. Use Pyomo to solve for the price per square foot in each year that maximizes the total earnings after tax. Use ipopt (since this is a nonlinear problem).

In [19]:
def office_earnings(total_sqft = 180000,
           m = -0.05,
           b = 1.5,
           op_expense_per_sqft = 1.20,
           heating_surcharge_per_sqft = .2,
           op_exp_annual_growth = .12,
           annual_mortgage = 1500000,
           tax_rate = .34,
           price_per_sqft = [15, 15, 15, 15, 15],
           num_years = 5):
  #rev calc
  perc_occ = [m*price_per_sqft[i] + b for i in range(num_years)]
  sqft_occ = [perc_occ[i]*total_sqft for i in range(num_years)]
  revenue = [sqft_occ[i]*price_per_sqft[i] for i in range(num_years)]
  #operating expense calculations
  base_op_cost_as_percY1 = [(1+op_exp_annual_growth)**i for i in range(num_years)] #note that range(num_years) = range(5) = [0,1,2,3,4] and (1+op_exp_annual_growth)**0 = 1.
  base_op_cost = [op_expense_per_sqft*total_sqft*base_op_cost_as_percY1[i] for i in range(num_years)]
  heating_surcharge = [perc_occ[i]*base_op_cost[i]*heating_surcharge_per_sqft for i in range(num_years)]
  mortgage = [annual_mortgage for i in range(num_years)]
  operating_costs = [base_op_cost[i] + heating_surcharge[i] + mortgage[i] for i in range(num_years)]
  #before and after-tax earnings
  ebt = [revenue[i] - operating_costs[i] for i in range(num_years)]
  taxes = [ebt[i]*tax_rate for i in range(num_years)]
  earnings_after_tax = [ebt[i] - taxes[i] for i in range(num_years)]
  total_earnings_after_tax = sum(earnings_after_tax)
  return total_earnings_after_tax

In [20]:
#initialize a "Concrete Model"
modelB = ConcreteModel()
#initialize DVs
num_years =5
modelB.price = Var(range(num_years),bounds=(0,100), initialize=15)
#define the objective
def objective_rule(model):
    price_list = [model.price[i] for i in range(num_years)]
    return office_earnings(price_per_sqft=price_list)
modelB.Objective = Objective(rule=objective_rule, sense = maximize)
#(Optional) You can use model.pprint() to see what you've done so far
modelB.pprint()


1 Var Declarations
    price : Size=5, Index={0, 1, 2, 3, 4}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :    15 :   100 : False : False :  Reals
          1 :     0 :    15 :   100 : False : False :  Reals
          2 :     0 :    15 :   100 : False : False :  Reals
          3 :     0 :    15 :   100 : False : False :  Reals
          4 :     0 :    15 :   100 : False : False :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : (-0.05*price[0] + 1.5)*180000*price[0] - (216000.0 + (-0.05*price[0] + 1.5)*216000.0*0.2 + 1500000) - ((-0.05*price[0] + 1.5)*180000*price[0] - (216000.0 + (-0.05*price[0] + 1.5)*216000.0*0.2 + 1500000))*0.34 + (-0.05*price[1] + 1.5)*180000*price[1] - (241920.00000000003 + (-0.05*price[1] + 1.5)*241920.00000000003*0.2 + 1500000) - ((-0.05*price[1] + 1.5)*180000*price[1] - (241920.00000000003 + (-0.05*price[1] + 1.5

In [21]:
#solve model (Note: you should use "ipopt" because this is a nonlinear problem)
opt = SolverFactory('ipopt')
results = opt.solve(modelB, tee = False) #setting tee = False hides the diagnostic outputs


In [22]:
#print optimal solution and the largest earnings it achieves
#print optimal solution and the largest earnings it achieves
print("Optimal Price Per Square Foot:")
for i in range(num_years):
    print(f"Year{i+1}: ${modelB.price[i].value:.2f}")
print(f"Total Earnings After Tax: ${modelB.Objective():.2f}")

Optimal Price Per Square Foot:
Year1: $15.12
Year2: $15.13
Year3: $15.15
Year4: $15.17
Year5: $15.19
Total Earnings After Tax: $691696.83


#Problem C: Coding with Lists of Lists and Constraint Lists
Solve this small version of the Stigler problem shown below using lists, `ConstraintLists`, and `for` loops. This version only has **4 decision variables (DVs)** and **3 constraints** but please code in a way that would be scalable for larger problems by following the structure I've started for you below. Print out the optimal \(x\)'s and the total optimal cost.

**Minimize cost =**  
$0.36*x_{\text{wheat}} + 0.141*x_{\text{mac}} + 0.242*x_{\text{cereal}} + 0.300*x_{\text{milk}}$

**Subject to:**  
$
16.1*x_{\text{wheat}} + 1.6*x_{\text{mac}} + 2.9*x_{\text{cereal}} + 12.5*x_{\text{milk}} \geq 3 \quad (\text{Calories Daily Min Constraint})
$
$
7.9*x_{\text{wheat}} + 58.9*x_{\text{mac}} + 91.2*x_{\text{cereal}} + 42.3*x_{\text{milk}} \geq 1.8 \quad (\text{Protein Daily Min Constraint})
$
$
80.5*x_{\text{wheat}} + 3.0*x_{\text{mac}} + 7.2*x_{\text{cereal}} + 15.4*x_{\text{milk}} \geq 2.5 \quad (\text{Fiber Daily Min Constraint})
$

$
x_{\text{wheat}}, x_{\text{mac}}, x_{\text{cereal}}, x_{\text{milk}} \geq 0
$


In [25]:
#I've put in these input parameters for you
num_commodities = 4    #this is how many food commodities to decide on
num_nutrients = 3      #this is how many nutrient constraints there are
cost_coef = [.36, 0.141, 0.242, 0.300]   #this is a list of the cost coefficients in the objective
constraint_coef = [[16.1, 1.6, 2.9, 12.5],
                   [7.9, 58.9, 91.2, 42.3],
                   [80.5, 3.0, 7.2, 15.4]]     #this is the list of lists of all the constraint coefficients
daily_mins = [3, 1.8, 2.5]        #these are the right hand sides

In [27]:
#Fill in the ??? to complete this code
modelC = ConcreteModel()

#dvs
modelC.x = Var(range(num_commodities), domain = NonNegativeReals)

#objective
modelC.Objective = Objective(expr = sum(cost_coef[i]*modelC.x[i] for i in range(num_commodities)), sense = minimize)

#constraints
modelC.nutrient_constraints = ConstraintList()
for j in range(num_nutrients):
  modelC.nutrient_constraints.add(expr=sum(constraint_coef[j][i] * modelC.x[i] for i in range(num_commodities))>= daily_mins[j])

#model pprint()
modelC.pprint()

1 Var Declarations
    x : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.36*x[0] + 0.141*x[1] + 0.242*x[2] + 0.3*x[3]

1 Constraint Declarations
    nutrient_constraints : Size=3, Index={1, 2, 3}, Active=True
        Key : Lower : Body                                         : Upper : Active
          1 :   3.0 :  16.1*x[0] + 1.6*x[1] + 2.9*x[2] + 12.5*x[3] :  +Inf :   True
          2 :   1.8 : 7.9*x[0] + 58.9*x[1] + 91.2*x[2] + 42.3*x[3] :  +Inf :   True
          3 :   2.5 :  80.5*x[0] + 3.0*x[1] + 7.2*x[2] +

In [28]:
#solve model with cbc because this is a linear program
opt = SolverFactory('cbc')
results = opt.solve(modelC, tee = True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmpwva8tqe_.pyomo.lp -stat=1 -solve -solu /tmp/tmpwva8tqe_.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 3 (0) rows, 4 (0) columns and 12 (0) elements
Statistics for presolved model


Problem has 3 rows, 4 columns (4 with objective) and 12 elements
Column breakdown:
4 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
3 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 3 (0) rows, 4 (0) columns and 12 (0) elements
0  Obj 0 Primal inf 0.23712785 (3)
2  Obj 0.067266607


In [29]:
#print optimal amounts of each food and the optimal cost it achieves
#print optimal amounts of each food and the optimal cost it achieves
print("Optimal Amounts of Each Food:")
for i in range(num_commodities):
    print(f"x{i}({['Wheat','Macaroni','Cereal','Milk'][i]}):{modelC.x[i].value:.4f}")
print(f"Total Optimal Cost is: ${modelC.Objective():.4f}")



Optimal Amounts of Each Food:
x0(Wheat):0.1793
x1(Macaroni):0.0000
x2(Cereal):0.0000
x3(Milk):0.0091
Total Optimal Cost is: $0.0673
